In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('HospInfo.csv')
df.head()

,Provider ID,Hospital Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership,...,Readmission national comparison footnote,Patient experience national comparison,Patient experience national comparison footnote,Effectiveness of care national comparison,Effectiveness of care national comparison footnote,Timeliness of care national comparison,Timeliness of care national comparison footnote,Efficient use of medical imaging national comparison,Efficient use of medical imaging national comparison footnote,Location
0,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,2565938310,Acute Care Hospitals,Government - Hospital District or Authority,...,NaN,Same as the national average,NaN,Same as the national average,NaN,Above the national average,NaN,Below the national average,NaN,"2505 U S HIGHWAY 431 NORTH\nBOAZ, AL\n"
1,10012,DEKALB REGIONAL MEDICAL CENTER,200 MED CENTER DRIVE,FORT PAYNE,AL,35968,DE KALB,2568453150,Acute Care Hospitals,Proprietary,...,NaN,Same as the national average,NaN,Same as the national average,NaN,Above the national average,NaN,Same as the national average,NaN,"200 MED CENTER DRIVE\nFORT PAYNE, AL\n"
2,10032,WEDOWEE HOSPITAL,209 NORTH MAIN STREET,WEDOWEE,AL,36278,RANDOLPH,2563572111,Acute Care Hospitals,Government - Hospital District or Authority,...,NaN,Not Available,Results are not available for this reporting p...,Same as the national average,NaN,Same as the national average,NaN,Not Available,Results are not available for this reporting p...,"209 NORTH MAIN STREET\nWEDOWEE, AL\n"
3,10095,HALE COUNTY HOSPITAL,508 GREEN STREET,GREENSBORO,AL,36744,HALE,3346243024,Acute Care Hospitals,Government - Local,...,Results are not available for this reporting p...,Not Available,Results are not available for this reporting p...,Below the national average,NaN,Same as the national average,NaN,Not Available,Results are not available for this reporting p...,"508 GREEN STREET\nGREENSBORO, AL\n"
4,10131,CRESTWOOD MEDICAL CENTER,ONE HOSPITAL DR SE,HUNTSVILLE,AL,35801,MADISON,2568823100,Acute Care Hospitals,Proprietary,...,NaN,Same as the national average,NaN,Same as the national average,NaN,Same as the national average,NaN,Same as the national average,NaN,"ONE HOSPITAL DR SE\nHUNTSVILLE, AL\n"


In [2]:
State = df[['State']].drop_duplicates()
State.columns=['StateName']
with open('State.csv','w') as f:
  f.write(State.to_csv(index=False,header=False))
State.index = list(range(1,len(State)+1))
State.head()

,StateName
1,AL
2,AK
3,AZ
4,AR
5,CA


In [3]:
County = df[['State','County Name']].drop_duplicates()
State2Id = dict(map(reversed, State.to_dict()['StateName'].items()))
County['State'] = County.apply(lambda x : State2Id[x['State']], axis=1)
County.index = list(range(1,len(County)+1))
County.columns = ['StateId','CountyName']
with open('County.csv','w') as f:
  f.write(County.to_csv(index=False,header=False))
County.head()

,StateId,CountyName
1,1,MARSHALL
2,1,DE KALB
3,1,RANDOLPH
4,1,HALE
5,1,MADISON


In [4]:
City = df[['County Name','City','State']].drop_duplicates()
County2Id = {(County.iloc[i-1]['CountyName'],County.iloc[i-1]['StateId']):i for i in range(1,len(County)+1)}
City['County Name'] = City.apply(lambda x : County2Id[(x['County Name'],State2Id[x['State']])], axis=1)
City = City.drop(columns=['State'])
City.index = list(range(1,len(City)+1))
City.columns = ['CountyId','CityName']
with open('City.csv','w') as f:
  f.write(City.to_csv(index=False,header=False))
City.head()

,CountyId,CityName
1,1,BOAZ
2,2,FORT PAYNE
3,3,WEDOWEE
4,4,GREENSBORO
5,5,HUNTSVILLE


In [5]:
Type = df[['Hospital Type']].drop_duplicates()
Type.columns=['HospitalType']
Type.index = list(range(1,len(Type)+1))
with open('Type.csv','w') as f:
  f.write(Type.to_csv(index=False,header=False))
Type.head()

,HospitalType
1,Acute Care Hospitals
2,Critical Access Hospitals
3,Childrens


In [7]:
Ownership = df[['Hospital Ownership']].drop_duplicates()
Ownership.columns=['HospitalOwnership']
Ownership.index = list(range(1,len(Ownership)+1))
with open('Ownership.csv','w') as f:
  f.write(Ownership.to_csv(index=False,header=False))
Ownership.head()

,HospitalOwnership
1,Government - Hospital District or Authority
2,Proprietary
3,Government - Local
4,Voluntary non-profit - Private
5,Tribal


In [8]:
Hospitals = df[['Hospital Type','Hospital Ownership','City','County Name','State','Hospital Name','Address',
        'ZIP Code','Phone Number','Emergency Services','Meets criteria for meaningful use of EHRs','Hospital overall rating']].drop_duplicates()
Hospitals.columns = ['Type','Ownership','City','County','State','Hospital_Name','Address',
        'ZIP_Code','Phone_Number','Emergency_Services','Efficient_EHR','Overall_Rating']

Type2Id = dict(map(reversed, Type.to_dict()['HospitalType'].items()))
Hospitals['Type'] = Hospitals.apply(lambda x : Type2Id[x['Type']], axis=1)

Ownership2Id = dict(map(reversed, Ownership.to_dict()['HospitalOwnership'].items()))
Hospitals['Ownership'] = Hospitals.apply(lambda x : Ownership2Id[x['Ownership']], axis=1)

City2Id = {(City.iloc[i-1]['CityName'],City.iloc[i-1]['CountyId']):i for i in range(1,len(City)+1)}
Hospitals['City'] = Hospitals.apply(lambda x : City2Id[(x['City'],County2Id[(x['County'],State2Id[x['State']])])], axis=1)
Hospitals = Hospitals.drop(columns=['County','State'])

def bool2int(b):
  if b == True:
    return 1
  elif b == False:
    return 0
  else:
    return b

Hospitals['Emergency_Services'] = Hospitals.apply(lambda x : bool2int(x['Emergency_Services']), axis=1)
Hospitals['Efficient_EHR'] = Hospitals.apply(lambda x : bool2int(x['Efficient_EHR']), axis=1)
Hospitals['Efficient_EHR'] = Hospitals['Efficient_EHR'].astype("Int64")

def rating2int(r):
  if len(r) == 1 and '0' <= r <= '9':
    return int(r)
  else:
    return None

Hospitals['Overall_Rating'] = Hospitals.apply(lambda x : rating2int(x['Overall_Rating']), axis=1)
Hospitals['Overall_Rating'] = Hospitals['Overall_Rating'].astype("Int64")

Hospitals.index = list(range(1,len(Hospitals)+1))
with open('Hospitals.csv','w') as f:
  f.write(Hospitals.to_csv(index=False,header=False))
Hospitals.head()

,Type,Ownership,City,Hospital_Name,Address,ZIP_Code,Phone_Number,Emergency_Services,Efficient_EHR,Overall_Rating
1,1,1,1,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,35957,2565938310,1,1,3
2,1,2,2,DEKALB REGIONAL MEDICAL CENTER,200 MED CENTER DRIVE,35968,2568453150,1,1,3
3,1,1,3,WEDOWEE HOSPITAL,209 NORTH MAIN STREET,36278,2563572111,1,1,4
4,1,3,4,HALE COUNTY HOSPITAL,508 GREEN STREET,36744,3346243024,1,1,NaN
5,1,2,5,CRESTWOOD MEDICAL CENTER,ONE HOSPITAL DR SE,35801,2568823100,1,1,3


In [9]:
Ratings = pd.DataFrame(data={'Rating_Name':['Below the national average','Same as the national average',
       'Above the national average']})
Ratings.index=[1,2,3]
with open('Ratings.csv','w') as f:
  f.write(Ratings.to_csv(index=False,header=False))
Ratings

,Rating_Name
1,Below the national average
2,Same as the national average
3,Above the national average


In [10]:
Criteria_Info = pd.DataFrame(data={'Criteria_Name':['Mortality national comparison',
    'Safety of care national comparison','Readmission national comparison',
    'Patient experience national comparison','Effectiveness of care national comparison',
    'Timeliness of care national comparison','Efficient use of medical imaging national comparison']})
Criteria_Info.index=list(range(1,8))
with open('Criteria_Info.csv','w') as f:
  f.write(Criteria_Info.to_csv(index=False,header=False))
Criteria_Info

,Criteria_Name
1,Mortality national comparison
2,Safety of care national comparison
3,Readmission national comparison
4,Patient experience national comparison
5,Effectiveness of care national comparison
6,Timeliness of care national comparison
7,Efficient use of medical imaging national comp...


In [11]:
Rating_Records = df[Criteria_Info['Criteria_Name']]
Rating_Records['Hospital_Id']= range(1,len(df)+1)
Rating_Records = pd.melt(Rating_Records, id_vars=['Hospital_Id'], value_vars=Criteria_Info['Criteria_Name'])
Rating_Records.drop(index = Rating_Records[Rating_Records['value'] == 'Not Available'].index, inplace=True)

Criteria2Id = dict(map(reversed, Criteria_Info.to_dict()['Criteria_Name'].items()))
Rating_Records['variable'] = Rating_Records.apply(lambda x: Criteria2Id[x['variable']], axis=1)

Rating2Id = dict(map(reversed, Ratings.to_dict()['Rating_Name'].items()))
Rating_Records['value'] = Rating_Records.apply(lambda x: Rating2Id[x['value']], axis=1)

Rating_Records.columns = ['Hospital_Id','Criteria_Id','Rating_Id']
Rating_Records.index = range(1,len(Rating_Records)+1)

with open('Rating_Records.csv','w') as f:
  f.write(Rating_Records.to_csv(index=False,header=False))

Rating_Records.head()

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Hospital_Id,Criteria_Id,Rating_Id
1,1,1,1
2,2,1,1
3,3,1,2
4,5,1,1
5,7,1,2


Then we just need to import from the generated csv files.